In [1]:
import torch
import torch.nn as nn
import scipy.optimize as optimize
import data.datasets as dataSource
import torch.utils.data as tdata
import numpy as np


Designer:
- mins and max in dataset, split dataset in two 

Here: 
- get Mask with "important_columns"
- MLP with batchnorm and initial batchnorm (should replace need for scaler)
- train + val loop for new network
- constraints from min, max
- constraints from designer.py in magnetdesigner
- project_to_polytope with optimize.minimize (https://stackoverflow.com/questions/17009774/quadratic-program-qp-solver-that-only-depends-on-numpy-scipy)
- input optim loop without optimizer:
    - loss_fun and tuple of interest as in other pynb
    - create x with requires_grad
    - for step in steps
        - x.grad.zero_()
        - for param in network param.grad.zero_()
        - forward, loss, backward
        - x = x - lr * x.grad
        - x = project_to_poly()
        - x.requires_grad_(True)    

In [2]:
dataset_train, dataset_val = dataSource.Dipole_H_train, dataSource.Dipole_H_val
important_cols = ["aper_x", "aper_y", "aper_x_tapering", "yoke_x", "yoke_y", "w", "w_leg", "totalCurrent"]
input_mask = [True if col in important_cols else False for col in dataset_train.input_columns]
train_loader, val_loader = tdata.DataLoader(dataset_train,64), tdata.DataLoader(dataset_val,64)

target_index_dict = {col:i for i,col in enumerate(dataset_train.target_columns)}


In [3]:
class MyRegNet(nn.Module):
    def __init__(self,h_size=32):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(len(important_cols), dtype=torch.double)
        self.fc1 = nn.Linear(len(important_cols),h_size, dtype=torch.double)
        self.nl1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm1d(h_size, dtype=torch.double)
        self.fc2 = nn.Linear(h_size,h_size, dtype=torch.double)
        self.nl2 = nn.LeakyReLU()
        self.bn2 = nn.BatchNorm1d(h_size, dtype=torch.double)
        self.fc3 = nn.Linear(h_size,33, dtype=torch.double)
    def forward(self,x):
        x = self.bn0(x)
        x = self.fc1(x)
        x = self.nl1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.nl2(x)
        x = self.bn2(x)
        x = self.fc3(x)
        return x

network = MyRegNet()

In [4]:
n_epochs = 100
lr = 1e-4
optim = torch.optim.Adam(network.parameters(), lr)
loss_fun = nn.MSELoss()

for epoch in range(n_epochs):
    network.train()
    loss_list = []
    for x,y in train_loader:
        selected_x = x[:,input_mask].double()
        pred = network(selected_x)
        loss = loss_fun(pred,y)
        loss_list.append(loss.detach().item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    epoch_train_loss = torch.tensor(loss_list).mean()

    network.eval()
    loss_list = []
    for x,y in val_loader:
        selected_x = x[:,input_mask].double()
        pred = network(selected_x)
        loss = loss_fun(pred,y)
        loss_list.append(loss.detach().item())
    epoch_val_loss = torch.tensor(loss_list).mean()
    print(epoch,epoch_train_loss,epoch_val_loss,end="\r")


KeyboardInterrupt: 

In [6]:
A = []
b = []

def onehot(name, neg):
    out = [0] * len(important_cols)
    out[important_cols.index(name)] = -1 if neg else 1
    return out


# constraints for minima: x > a --> -x < -a
for name,value in zip(important_cols,dataset_train.input_mins):
    A.append(onehot(name,True))
    b.append(-value)

# constraints for maxima: x < a
for name,value in zip(important_cols,dataset_train.input_mins):
    A.append(onehot(name,False))
    b.append(value)

In [7]:
A_rel = []
b_rel = []

# "aper_x", "aper_y", "aper_x_poleoverhang", "aper_x_tapering", "yoke_x", 
# "yoke_y", "w"     , "w_leg"              , "totalCurrent"]


# aper_x > aper_y --> aper_y - aper_x < 0
A_rel.append([-1, 1, 0, 0, 0, 0, 0, 0, 0])
b_rel.append(0)

# aper_x < 5 * aper_y --> aper_x - 5 * aper_y < 0
A_rel.append([1, 5, 0, 0, 0, 0, 0, 0, 0])
b_rel.append(0)

# w_leg < aper_x --> w_leg - aper_x < 0
A_rel.append([-1, 0, 0, 0, 0, 0, 0, 1, 0])
b_rel.append(0)

## Max-made
# w < 2*w_leg --> w - 2w_leg < 0
A_rel.append([ 0, 0, 0, 0, 0, 0, 1,-2, 0])
b_rel.append(0)

# w_leg < w --> w_leg - 2w < 0
A_rel.append([ 0, 0, 0, 0, 0, 0,-2, 1, 0])
b_rel.append(0)

# make coils squareish (aspect ratio of no more than 2 --> for sides a and be we have a - 2b < 0 and b - 2a < 0)
# coil width = 0.5 yoke_x - 0.5 w - aper_tape - w_leg
# coil height = 0.5 yoke_y - 0.6 aper_y - w_leg

# coil_width - 2 coil_height < 0
A_rel.append([ 0, 1.2, 0, -1, 0.5, -1, -0.5, 1, 0])
b_rel.append(0)

# coil_height - 2 coil_width < 0
A_rel.append([ 0, -0.6, 0, 2, -1, 0.5, 1, 1, 0])
b_rel.append(0)

In [8]:
A = A + A_rel
b = b + b_rel

In [9]:
def project_to_poly(x, A_mat=A,b_vec = b):
    def loss(curr_guess):
        return np.linalg.norm(curr_guess-x)
    cons = {'type':'ineq','fun':lambda x: b_vec - np.dot(A_mat,x)}
    return optimize.minimize(loss, x,constraints=cons, method='SLSQP').x

#A_test = [[0,1],[1,0]]
#b_test = [1,1]
#project_to_poly([2,0.1],A_test,b_test)

In [10]:
tuple_of_interest = (target_index_dict["B0"],target_index_dict["gfr_x_5e-3"],target_index_dict["gfr_y_5e-3"])

target = torch.zeros(1,33)
target[0,tuple_of_interest[0]] = 0.079589
target[0,tuple_of_interest[1]] = 0.053487
target[0,tuple_of_interest[2]] = 0.01

def create_loss_function(tuple_of_interest):
    mask = torch.zeros(1,33)
    for i in tuple_of_interest:
        mask[0,i] = 1
    mse = torch.nn.MSELoss()
    def instanciated_loss_function(x,y):
        return mse(x*mask,y*mask)
    return instanciated_loss_function

loss_fun = create_loss_function(tuple_of_interest)

In [18]:
network.bn0.weight

Parameter containing:
tensor([0.6861, 1.1442, 0.9570, 0.9384, 1.0204, 1.0346, 0.8320, 0.9992],
       dtype=torch.float64, requires_grad=True)

In [1]:
magnet_params = torch.randn((1,9),dtype=torch.double,requires_grad=True)

lr = 1e-3

network.eval()

n_steps = 1000

for step in range(n_steps):
    magnet_params.grad.zero_()
    for layer in network.parameters():
        layer.grad.zero_()
    
    pred = network(magnet_params)
    loss = loss_fun(pred,target)
    loss.backward()

    with torch.no_grad():
        magnet_params = magnet_params - 


NameError: name 'torch' is not defined